# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset - mongoimport --type json -d uk_food -c establishments --headerline --drop establishments.json

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
pprint(mongo.list_database_names())

['GardenDB',
 'admin',
 'autosaurus',
 'config',
 'epa',
 'local',
 'met',
 'petsitly_marketing',
 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
pprint(db.list_collection_names())

['establishments']


In [8]:
# assign the collection to a variable
establishments = db['establishments']

In [9]:
# review a document in the establishments collection
document = establishments.find_one()
pprint(document)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66904ac2492bd368debb91a1'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [11]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('66904cbfa5bc65ab517dec01'), acknowledged=True)

In [12]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66904cbfa5bc65ab517dec01'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [13]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {"BusinessName": 0, "AddressLine1": 0,"AddressLine2": 0,"AddressLine3": 0,"AddressLine4": 0,
         "PostCode": 0,"Phone": 0,"LocalAuthorityCode": 0,"LocalAuthorityName": 0, "LocalAuthorityWebSite": 0,"LocalAuthorityEmailAddress": 0,
         "scores": 0,"SchemeType": 0,"geocode": 0,"RightToReply": 0,"Distance": 0,"NewRatingPending": 0,"FHRSID": 0,"ChangesByServerID": 0,
         "LocalAuthorityBusinessID": 0,"RatingValue": 0,"RatingKey": 0,"RatingDate": 0,"meta": 0,"links": 0,"_id": 0}
results = establishments.find(query, fields)
df = pd.DataFrame(results)
df.head(10)

,BusinessType,BusinessTypeID
0,Restaurant/Cafe/Canteen,1
1,Restaurant/Cafe/Canteen,1
2,Restaurant/Cafe/Canteen,1
3,Restaurant/Cafe/Canteen,1
4,Restaurant/Cafe/Canteen,1
5,Restaurant/Cafe/Canteen,1
6,Restaurant/Cafe/Canteen,1
7,Restaurant/Cafe/Canteen,1
8,Restaurant/Cafe/Canteen,1
9,Restaurant/Cafe/Canteen,1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [14]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one({"BusinessName": "Penang Flavours"}, {"$set":{"BusinessTypeID":1}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [16]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
fields = {"BusinessType": 0, "AddressLine1": 0,"AddressLine2": 0,"AddressLine3": 0,"AddressLine4": 0,
         "PostCode": 0,"Phone": 0,"LocalAuthorityCode": 0,"LocalAuthorityName": 0, "LocalAuthorityWebSite": 0,"LocalAuthorityEmailAddress": 0,
         "scores": 0,"SchemeType": 0,"geocode": 0,"RightToReply": 0,"Distance": 0,"NewRatingPending": 0,"FHRSID": 0,"ChangesByServerID": 0,
         "LocalAuthorityBusinessID": 0,"RatingValue": 0,"RatingKey": 0,"RatingDate": 0,"meta": 0,"links": 0,"_id": 0}
results = establishments.find(query, fields)
df = pd.DataFrame(results)
df.head()

,BusinessName,BusinessTypeID
0,Penang Flavours,1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [17]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName":"Dover"}
establishments.count_documents(query)

994

In [18]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName":"Dover"}
establishments.delete_many(query)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [19]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName":"Dover"}
establishments.count_documents(query)

0

In [20]:
# Check that other documents remain with 'find_one'
other_documents = establishments.find_one()
pprint(other_documents)


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66904ac2492bd368debb9488'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [40]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}} ])
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}} ])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [41]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [42]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
])

UpdateResult({'n': 38786, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [37]:
# Check that the coordinates and rating value are now numbers
results = establishments.find().limit(10)

# Iterate over the results and check the data types of 'latitude', 'longitude', and 'RatingValue'
for doc in results:
    latitude_type = type(doc.get('latitude'))
    longitude_type = type(doc.get('longitude'))
    rating_value_type = type(doc.get('RatingValue'))
    pprint({
        'Latitude data type': latitude_type,
        'Longitude data type': longitude_type,
        'RatingValue data type': rating_value_type
    })

{'Latitude data type': <class 'NoneType'>,
 'Longitude data type': <class 'NoneType'>,
 'RatingValue data type': <class 'int'>}
{'Latitude data type': <class 'NoneType'>,
 'Longitude data type': <class 'NoneType'>,
 'RatingValue data type': <class 'int'>}
{'Latitude data type': <class 'NoneType'>,
 'Longitude data type': <class 'NoneType'>,
 'RatingValue data type': <class 'int'>}
{'Latitude data type': <class 'NoneType'>,
 'Longitude data type': <class 'NoneType'>,
 'RatingValue data type': <class 'int'>}
{'Latitude data type': <class 'NoneType'>,
 'Longitude data type': <class 'NoneType'>,
 'RatingValue data type': <class 'int'>}
{'Latitude data type': <class 'NoneType'>,
 'Longitude data type': <class 'NoneType'>,
 'RatingValue data type': <class 'int'>}
{'Latitude data type': <class 'NoneType'>,
 'Longitude data type': <class 'NoneType'>,
 'RatingValue data type': <class 'int'>}
{'Latitude data type': <class 'NoneType'>,
 'Longitude data type': <class 'NoneType'>,
 'RatingValue dat